In [1]:
from bs4 import BeautifulSoup 
import re 
import requests
import os
import nltk 
from sklearn import feature_extraction
from gensim import models
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding, Bidirectional
from keras.callbacks import ModelCheckpoint, LambdaCallback

/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/sklearn/utils/fixes.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if 'order' in inspect.getargspec(np.copy)[0]:
/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# Faz uma requisição na página que contém mil links de letras de músicas sertanejas
url_musicas = "https://www.letras.mus.br/mais-acessadas/sertanejo/"
pagina = requests.get(url_musicas)

In [ ]:
# Guarda o conteúdo da página
conteudo = pagina.content

In [ ]:
# Cria o parser HTML
soup = BeautifulSoup(conteudo,"html.parser")

In [ ]:
top_musicas = soup.find("ol",attrs={"class" : "top-list_mus"})

In [ ]:
links_musicas = top_musicas.findAll("a")

In [ ]:
links_musicas[0]

In [ ]:
raiz_site = "https://www.letras.mus.br/"
for link in links_musicas:
    pagina_musica = requests.get(raiz_site + link.get("href"))
    soup_musica = BeautifulSoup(pagina_musica.content,"html.parser")
    artigo_letra = soup_musica.find("article")
    nome_arquivo = str(link.get("href")).replace("/","") + ".html"
    with open("dados/letras-musicas/" + nome_arquivo,"w") as f:
        f.write(str(artigo_letra))

In [2]:
documentos = []
arquivos = os.listdir("dados/letras-musicas")
for arquivo in arquivos:
    caminho = "dados/letras-musicas/" + arquivo
    if os.path.isfile(caminho):
        with open(caminho,"r") as f:
            html = f.read()
        html = feature_extraction.text.strip_accents_ascii(html.lower())
        html = html.replace("<br/>"," NLINHA ")
        html = html.replace("<p>"," NLINHA ")
        soup = BeautifulSoup(html,"html.parser")
        documentos.append(soup.text.strip())

In [3]:
docs_tokenizados = []
tokenizador = nltk.TreebankWordTokenizer()
for doc in documentos:
    tokens = tokenizador.tokenize(doc)
    docs_tokenizados.append(tokens)

In [4]:
docs_tokenizados[0]

['NLINHA',
 'garcom',
 'me',
 'traga',
 'NLINHA',
 'outra',
 'garrafa',
 'de',
 'cerveja',
 'NLINHA',
 'vou',
 'ficar',
 'sozinho',
 'nessa',
 'mesa',
 'NLINHA',
 'eu',
 'quero',
 'beber',
 'NLINHA',
 'e',
 'chorar',
 'por',
 'ela',
 '...',
 'NLINHA',
 'garcom',
 'a',
 'minha',
 'vida',
 'agora',
 'NLINHA',
 'ta',
 'de',
 'ponta',
 'cabeca',
 'NLINHA',
 'ja',
 'tentei',
 'mas',
 'nada',
 'NLINHA',
 'faz',
 'com',
 'que',
 'eu',
 'esqueca',
 'NLINHA',
 'os',
 'olhos',
 'e',
 'os',
 'labios',
 'NLINHA',
 'daquela',
 'mulher',
 '...',
 'NLINHA',
 'garcom',
 'ela',
 'saiu',
 'de',
 'vez',
 'NLINHA',
 'da',
 'minha',
 'vida',
 'NLINHA',
 'e',
 'agora',
 'eu',
 'busco',
 'uma',
 'saida',
 'NLINHA',
 'minha',
 'historia',
 'de',
 'amor',
 'NLINHA',
 'acaba',
 'em',
 'solidao',
 'NLINHA',
 'garcom',
 'se',
 'eu',
 'ficar',
 'NLINHA',
 'muito',
 'chato',
 'NLINHA',
 'e',
 'der',
 'algum',
 'vexame',
 'NLINHA',
 'pegue',
 'toda',
 'a',
 'minha',
 'NLINHA',
 'cerveja',
 'e',
 'derrame',
 'NLINHA'

In [16]:
# Treinando um modelo Word2vec nos documentos
w2v_model = models.Word2Vec(docs_tokenizados, size=350, window=5, min_count=0, workers=os.cpu_count(),iter=300)

In [17]:
w2v_model.wv["amor"]

array([ 0.3406019 , -1.9125925 ,  1.1969016 , -0.5587075 , -0.70218074,
       -2.1437657 , -2.4969337 ,  0.97440696,  0.29571   , -1.089607  ,
       -1.0300038 , -0.48746398,  1.2007291 ,  0.419303  ,  0.5662048 ,
        2.2163913 ,  0.7217492 ,  0.80493313,  0.04219851,  0.00667225,
        0.04579617,  0.5727392 , -2.4367058 , -0.36421248, -1.9121523 ,
        0.15311125,  1.584396  , -0.42053974, -0.27861053,  0.8368535 ,
       -1.2631261 ,  1.5685248 ,  0.625242  ,  1.2720928 , -0.4132451 ,
        1.3151213 , -0.6245615 , -2.0052886 , -1.1362255 ,  0.9042989 ,
       -0.40036657,  3.8082623 , -0.8155124 , -0.36612707,  0.90110785,
        0.96860087,  0.4026807 ,  0.9611812 , -0.87658   , -0.3384384 ,
       -1.381545  , -1.382255  , -0.688124  , -1.6396114 , -0.4570476 ,
        0.5046095 , -0.7664078 ,  0.47183067, -0.1648448 , -0.82192135,
       -0.7883782 ,  0.70848244, -0.8908535 ,  1.4560546 , -1.5644749 ,
        1.0454316 , -0.6331501 ,  0.64759076, -0.8398884 ,  0.66

In [18]:
# Cria um array com o vetor das duas mil primeiras palavras
vocab = list(w2v_model.wv.vocab)
vetores_w2v = w2v_model.wv[vocab]

In [19]:
import numpy as np
vetores_w2v = np.asfarray(vetores_w2v)

In [ ]:
# Reduzindo a dimensionalidade dos vetores das palavras
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
vetores_reduzidos = tsne.fit_transform(vetores_w2v)

In [ ]:
# Cria um dataframe com as colunas x e y. O objetivo é transformar cada representação de uma palavra em uma coordenada.
import pandas as pd
df = pd.DataFrame(vetores_reduzidos, index=vocab, columns=['x', 'y'])
df.head()

In [ ]:
# Utiliza o matplotlib com a chave de interação ligada.
%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for word, pos in df.iterrows():
    ax.annotate(word, pos) # Coloca a palavra em cada ponto

ax.scatter(df['x'], df['y']) # Cria um scatterPlot com todos os pontos x e y

In [ ]:
w2v_model.save("modelos/w2v_sertanejo_1000musicas.model")

In [20]:
w2v_model.wv.vectors.shape

(8442, 350)

In [21]:
# Funcoes que retornam o índice da palavra no vocabulário criado pelo word2vec e vice-versa
def word2idx(word):
  return w2v_model.wv.vocab[word].index

def idx2word(idx):
  return w2v_model.wv.index2word[idx]

In [22]:
maxlen = 5
sentences = []
next_tokens = []

for doc in docs_tokenizados: # Cria as "sentencas" para cada documento do corpus
    for i in range(0,len(doc) - maxlen):
        sentences.append(doc[i:i+maxlen])
        next_tokens.append(doc[i+maxlen])

print('Number of sequences:', len(sentences), "\n")

# sentences = sentences[0:30000] # Limitando em 30 mil sentenças para não demorar...
# next_tokens = next_tokens[0:30000]

Number of sequences: 223804 



In [23]:
# Criando os inputs(x) e targets(y)
x = np.zeros((len(sentences), maxlen), dtype=np.int32)
y = np.zeros((len(sentences)), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, token in enumerate(sentence):
        x[i, t] = word2idx(token) 
    y[i] = word2idx(next_tokens[i])

In [25]:
pretrained_weights = w2v_model.wv.vectors

In [26]:
tamanho_vocab = pretrained_weights.shape[0]
tamanho_vetor_w2v = pretrained_weights.shape[1] # 350
print("Tamanho vocab e w2v vector: ", (tamanho_vocab, tamanho_vetor_w2v))

Tamanho vocab e w2v vector:  (8442, 350)


In [79]:
# Definindo o modelo LSTM
model = Sequential()
# A cama de de Embedding é onde podemos passar para a LSTM os vetores já treinados pelo word2vec
model.add(Embedding(input_dim= tamanho_vocab, output_dim=tamanho_vetor_w2v, weights=[pretrained_weights]))
model.add(Bidirectional(LSTM(350, activation="relu", return_sequences=True)))
model.add(Dropout(0.1))
model.add(LSTM(units=280))
# model.add(LSTM(64, input_shape=(pretrained_weights.shape[0], pretrained_weights.shape[1]) ))
# model.add(Dropout(0.1))
model.add(Dense(tamanho_vocab, activation='softmax')) # Quantidade de 'respostas' possíveis. Tokens neste caso. 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [74]:
# define the checkpoint
caminho_modelo_lstm ="modelos/bilstm-w2v-wordlevel-350-280-sertanejo.model"
checkpoint = ModelCheckpoint(caminho_modelo_lstm, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [75]:
def gerar_texto(epoch, logs):
    
    print('----- Generating text after Epoch: %d' % epoch)
    
    # pick a random seed
    start = np.random.randint(0, len(sentences))
    seed_tokens = list(sentences[start])
    print ("Seed:")
    print ("\"", " ".join(seed_tokens), "\"\n")
    
    # generate characters
    for i in range(500):
        
        xt = np.zeros((1, maxlen), dtype=np.int32)
        
        for t, token in enumerate(seed_tokens):
            xt[0,t] = word2idx(token)
        
        prediction = model.predict(xt, verbose=0)
#         ordered = (-prediction[0]).argsort()[:1]
#         print(len(ordered),ordered)
#         for x in ordered:
#             print(prediction[0][x])
#         indice_aleatorio = np.random.choice(ordered)
        index = np.argmax(prediction)
        result = idx2word(index)
        print("\n" if result == "NLINHA" else result,end=" ")
        seed_tokens.append(result)
        seed_tokens = seed_tokens[1:len(seed_tokens)]
        
    print("\n\nFIM\n\n",)

In [76]:
# Callback para gerar texto de exemplo ao final de cada iteração de treino
print_callback = LambdaCallback(on_epoch_end=gerar_texto)
callbacks_list.append(print_callback)

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 350)         2954700   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 360)               764640    
_________________________________________________________________
dropout_13 (Dropout)         (None, 360)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 8442)              3047562   
Total params: 6,766,902
Trainable params: 6,766,902
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.fit(x, y, epochs=30, batch_size=64, callbacks=callbacks_list)

Epoch 1/30
223804/223804 [==============================] - 475s 2ms/step - loss: 4.8188

Epoch 00001: loss improved from inf to 4.81885, saving model to modelos/bilstm-w2v-wordlevel-180-sertanejo.model
----- Generating text after Epoch: 0
Seed:
" ta ai em casa ? "


 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu coracao 
 e o meu cor

 a lua iluminando o sol 
 quero acordar todo dia 
 pra te fazer todo mundo 
 eu quero ver voce 
 eu quero te ver 
 eu quero ser pra voce 
 a lua iluminando o sol 
 quero acordar todo dia 
 pra te fazer todo mundo 
 eu quero ver voce 
 eu quero te ver 
 eu quero ser pra voce 
 a lua iluminando o sol 
 quero acordar todo dia 
 pra te fazer todo mundo 
 eu quero ver voce 
 eu quero te ver 
 eu quero ser pra voce 
 a lua iluminando o sol 
 quero acordar todo dia 
 pra te fazer todo mundo 
 eu quero ver voce 
 eu quero te ver 
 eu quero ser pra voce 
 a lua iluminando o sol 
 quero acordar todo dia 
 pra te fazer todo mundo 
 eu quero ver voce 
 eu quero te ver 
 eu quero ser 

FIM


Epoch 5/30
223804/223804 [==============================] - 842s 4ms/step - loss: 1.8560

Epoch 00005: loss improved from 2.18216 to 1.85604, saving model to modelos/bilstm-w2v-wordlevel-180-sertanejo.model
----- Generating text after Epoch: 4
Seed:
" NLINHA aproveitar a noite , "

ficar louca 
 e o seu coracao

 perdendo tempo 
 e ai voce sorriu 
 e me abracou 
 o cobertor que o sonho e amor que eu canto 
 felicidade muitos anos de amor 
 parabens a voce mesmo 
 e so porque esse sorriso lindo 
 voce me da pra mim 
 sou da vida 
 coitada , ja passou da minha cama 
 to valendo 
 o amor nao fosse descartavel 
 o meu corpo da estrada 
 duvido que vai me dar mais uma chance de olhar 
 que eu fui feito uma licao 
 e so pra te lembrar 
 a gente se perguntar nao 
 e quer saber ? que ce deixa ta 
 ce gosta de mim para de graca 
 oi , vem 
 meu peito , vem 
 e nao me importa o que vao dizer 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 
 eu quero so voce 


 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a 

FIM


Epoch 12/30
223804/223804 [===

 vou te cobrar com juros o que voce me tirou 
 se ate o tempo passa , imagina o seu amor ? 
 devolve o meu tempo 
 que eu fiquei ligando , mandando mensagem 
 implorando por favor 
 vou te cobrar com juros o que voce me tirou 
 se ate o tempo passa , imagina o seu amor ? 
 devolve o meu tempo 
 que eu fiquei ligando , mandando mensagem 
 implorando por favor 
 vou te cobrar com juros o que voce me tirou 
 se ate o tempo passa , imagina o seu amor ? 
 devolve o meu tempo 
 que eu fiquei ligando , mandando mensagem 
 implorando por favor 
 vou te cobrar com juros o que voce me tirou 
 se ate o tempo passa , imagina o seu amor ? 
 devolve o meu tempo 
 que eu fiquei ligando , mandando mensagem 
 implorando por favor 
 vou te cobrar com juros o que voce me tirou 
 se ate o tempo passa , imagina o seu amor ? 
 devolve o meu tempo 
 que eu fiquei ligando , mandando mensagem 
 implorando por favor 
 vou te cobrar com juros o que voce me tirou 
 se ate o tempo passa , imagina o seu amor ? 
 de

 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche , tche , tche 
 gusttavo lima e voce 
 tche tcherere tche tche 
 tcherere tche tche 
 tchereretche 
 tche 

FIM


Epoch 19/30
223804/223804 [==============================] - 630s 3ms/step - loss: 0.9865

Epoch 00019: loss improved from 

----- Generating text after Epoch: 21
Seed:
" eu te desejo NLINHA nao "

consigo te esquecer 
 a carne e fraca , coracao e vagabundo 
 e mesmo assim ainda bebo o seu 
 e depois de nos casados 
 ele vai mudando de alguem 
 vai ver o que perdeu quando me encontrar 
 nos bracos de outro alguem 
 ai ce vai chorar 
 e vai ser bem mais 
 e que eu me atrapaio 
 eu entro na venda e ja dou meu taio 
 pego no copo e dali nun distante 
 eu nao me pergunto : o que foi que fiz ? 
 mas sao palavras que ninguem responde 
 te vejo infeliz 
 voce deve estar com medo de contar 
 que seu amor , por mim acabou , sempre fingiu , nada sentiu 
 na hora de amar 
 a qualquer hora de mim 
 mas vai ouvir voce que o melhor hora 
 agora ela e que esse amor tem que entender ? 
 me deu aula de como aprender te esquecer 
 foi , mas ja chorei 
 hoje bem de sua porta 
 eu digo 
 a viola , fui sentindo o sol que eu to procurando 
 como eu nao to 
 nem me tira a manha 
 o abraco , nem me amar 
 nem na sua decisao 
 nem d

223804/223804 [==============================] - 673s 3ms/step - loss: 0.9279

Epoch 00026: loss improved from 0.93679 to 0.92793, saving model to modelos/bilstm-w2v-wordlevel-180-sertanejo.model
----- Generating text after Epoch: 25
Seed:
" NLINHA virou meia noite e "

a primeira 
 mas eu tive a pena 
 que o amor nao deu pra nao 
 e se quiser dizer : 
 e bem , que eu to aqui 
 eu sei que nao tem jeito 
 so quero voce pode lembrar de mim 
 que toda vida eu te passado 
 e agora me abraca 
 voce me fez assim um sonho 
 que so preciso te dizer 
 pra amar voce 
 quando voce e minha alegria 
 nem me da esse dia 
 o nosso amor e que a felicidade foi a felicidade 
 e a hora e agora , a hora e agora 
 desliga esse abajur 
 que o show nao precisa de luz 
 acao , romance ou suspense 
 a escolha e sua 
 mas pensando bem , me sinto um beijo 
 me ter pra poder te amar 
 ela e sua 
 mas o tempo passa cai 
 e essa noite se quer 
 eu sou seu 
 e a forca que e que nao tem nas outras que tem em voce 
 v

 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto , tem briga 
 quando a gente se separa , saudade 
 quando marca um encontro , discute 
 desconheco um amor tao covarde 
 quando a gente fica junto 

FIM


Epoch 30/30
2

In [ ]:
gerar_texto(1,"teste")